## EJERCICIO 4B - María Carrasco Meléndez

#### 1. Mínimo, máximo y media del tamaño de las peticiones.
#### 2. Número de peticiones de cada código de respuesta (response_code)
#### 3.  Mostrar 20 hosts que han sido visitados más de 10 veces.
#### 4. Mostrar los 10 endpoints más visitados.
#### 5. Mostrar los 10 endpoints más visitados que no tienen código de respuesta =200.
#### 6. Calcular el número de hosts distintos.
#### 7. Calcular el número de hosts únicos cada día.
#### 8. Calcular la media de peticiones diarias por host.
#### 9. Mostrar una lista de 40 endpoints distintos que generan un código respuesta = 404.
#### 10. Mostrar el top 25 de endpoints que más códigos de respuesta 404 generan.
#### 11. El top 5 de días que se generaron código de respuesta 404.


### En primer lugar para poder responder a estas preguntas tendremos que parsear nuestro dataset.

In [61]:
from pyspark import SparkContext
sc = SparkContext("local", "First App")

In [62]:
from pyspark.sql import SQLContext, Row # Row nos permite darle nombres a cada elemento de la lista
sqlContext = SQLContext(sc)

In [63]:
data_file = "./apache.access.log_small.txt"
raw_data = sc.textFile(data_file).cache()

Hasta aquí el cargado de datos

In [65]:
import re
def parse_log2(line):
    match = re.search('^(\S+) (\S+) (\S+) \[(\S+) [-](\d{4})\] "(\S+)\s*(\S+)\s*(\S+)\s*([/\w\.\s*]+)?\s*"* (\d{3}) (\S+)',line)
    if match is None:
        match = re.search('^(\S+) (\S+) (\S+) \[(\S+) [-](\d{4})\] "(\S+)\s*([/\w\.]+)>*([\w/\s\.]+)\s*(\S+)\s*(\d{3})\s*(\S+)',line)
    if match is None:
        return (line, 0)
    else:
        return (line, 1)

In [66]:
def map_log(line):
    match = re.search('^(\S+) (\S+) (\S+) \[(\S+) [-](\d{4})\] "(\S+)\s*(\S+)\s*(\S+)\s*([/\w\.\s*]+)?\s*"* (\d{3}) (\S+)',line)
    if match is None:
        match = re.search('^(\S+) (\S+) (\S+) \[(\S+) [-](\d{4})\] "(\S+)\s*([/\w\.]+)>*([\w/\s\.]+)\s*(\S+)\s*(\d{3})\s*(\S+)',line)
    return(match.groups())
parsed_rdd = raw_data.map(lambda line: parse_log2(line)).filter(lambda line: line[1] == 1).map(lambda line : line[0]) #map con el segundo parseador, filtramos cuales de ellos han sido uno (que el paseador los pasa adecuadamente) y despues mapeamos
parsed_def = parsed_rdd.map(lambda line: map_log(line))
parsed_def.take(1)

[('in24.inetnebr.com',
  '-',
  '-',
  '01/Aug/1995:00:00:01',
  '0400',
  'GET',
  '/shuttle/missions/sts-68/news/sts-68-mcc-05.txt',
  'HTTP/1.0"',
  None,
  '200',
  '1839')]

In [67]:
def convert_long(x):
    x = re.sub('[^0-9]',"",x)
    if x == "":
        return 0
    else:
        return int(x)
    

In [68]:
sql_data = parsed_def.map(lambda p: 
                         Row(Host = p[0], 
                            date = p[3][:2],
                            endpoint = p[6], 
                            codigo = p[9],
                            size = convert_long(p[10])
                            )
                         )
sql_data.count()

3432

In [69]:
data_df = sqlContext.createDataFrame(sql_data)
data_df.registerTempTable("data")

In [70]:
df_data = sqlContext.sql("""
    SELECT * FROM data LIMIT 20
""")

#### 1. Mínimo, máximo y media del tamaño de las peticiones.

In [71]:
df_data2 = sqlContext.sql("""
    SELECT MIN(size) AS minimo, MAX(size) AS maximo, AVG(size) AS media FROM data
""")

In [74]:
df_data2.show()

+------+------+------------------+
|minimo|maximo|             media|
+------+------+------------------+
|     0|887988|16051.863636363636|
+------+------+------------------+



#### 2. Número de peticiones de cada código de respuesta (response_code)

In [75]:
df_data3 = sqlContext.sql("""
    SELECT SUM(size) AS peticiones, codigo FROM data GROUP BY codigo
""")
df_data3.show()

+----------+------+
|peticiones|codigo|
+----------+------+
|  55085536|   200|
|      4460|   302|
|         0|   404|
|         0|   403|
|         0|   304|
+----------+------+



#### 3.  Mostrar 20 hosts que han sido visitados más de 10 veces.

In [92]:
df_data4 = sqlContext.sql("""
    SELECT COUNT (host) FROM data GROUP BY host HAVING COUNT(host)>10 LIMIT 20
""")
df_data4.show()

+-----------+
|count(host)|
+-----------+
|         15|
|         14|
|         12|
|         55|
|         12|
|         11|
|         28|
|         78|
|         17|
|         16|
|         29|
|         31|
|         14|
|         17|
|         17|
|         32|
|         13|
|         40|
|         19|
|         17|
+-----------+



#### 4. Mostrar los 10 endpoints más visitados.

In [117]:
df_data5 = sqlContext.sql("""
    SELECT COUNT (endpoint) FROM data GROUP BY endpoint ORDER BY COUNT(endpoint) DESC LIMIT 10
""")
df_data5.show()

#En primer lugar empleamos un GROUP BY para poder ordenar por endpoint limitando a 10, tras esto se le aplica el ORDER BY
# Donde ordenamos en orden descendente. 

+---------------+
|count(endpoint)|
+---------------+
|            167|
|            160|
|            122|
|            120|
|            118|
|            106|
|             85|
|             74|
|             69|
|             66|
+---------------+



#### 5. Mostrar los 10 endpoints más visitados que no tienen código de respuesta =200.

In [106]:
df_data6 = sqlContext.sql("""
    SELECT endpoint, codigo FROM data WHERE codigo!=200 ORDER BY endpoint DESC LIMIT 10
""")
df_data6.show()

+--------------------+------+
|            endpoint|codigo|
+--------------------+------+
|/www/software/win...|   404|
|     /whats-new.html|   304|
|     /whats-new.html|   304|
|     /sts-71/launch/|   404|
|/software/winvn/w...|   304|
|/software/winvn/w...|   304|
|/software/winvn/w...|   304|
|/software/winvn/w...|   304|
|/software/winvn/w...|   304|
|/software/winvn/w...|   304|
+--------------------+------+



#### 6. Calcular el número de hosts distintos.

In [79]:
df_data7 = sqlContext.sql("""
    SELECT COUNT (DISTINCT host) FROM data
""")
df_data7.show()

+--------------------+
|count(DISTINCT host)|
+--------------------+
|                 311|
+--------------------+



#### 7. Calcular el número de hosts únicos cada día.

In [81]:
df_data8 = sqlContext.sql("""
    SELECT COUNT (host), date FROM data GROUP BY date
""")
df_data8.show()

+-----------+----+
|count(host)|date|
+-----------+----+
|       3432|  01|
+-----------+----+



#### 8. Calcular la media de peticiones diarias por host.

In [108]:
df_data9 = sqlContext.sql("""
    SELECT AVG(size) AS media, date, host FROM data GROUP BY date,host
""")
df_data9.show()

+------------------+----+--------------------+
|             media|date|                host|
+------------------+----+--------------------+
|           39762.6|  01|sunset.somcau.siu...|
|           15974.0|  01|    igate.uswest.com|
|            2191.0|  01|kgtyk4.kj.yamagat...|
| 21095.53846153846|  01|    wwwproxy.info.au|
| 4960.774193548387|  01|       193.84.66.147|
|14210.416666666666|  01|dialup12.leuven.e...|
| 3448.818181818182|  01|dd15-053.compuser...|
| 4184.210526315789|  01|miriworld.its.uni...|
|3246.8333333333335|  01|ix-esc-ca2-07.ix....|
|            4287.0|  01|      133.102.242.50|
|            3640.0|  01| tyo1.gate.nec.co.jp|
| 8159.142857142857|  01|ix-dgr-il1-15.ix....|
|15684.777777777777|  01|      198.248.59.123|
|          6085.075|  01|hsccs_gatorbox07....|
|            2533.0|  01|mars.sunmtm.kuleu...|
| 7508.952380952381|  01|piweba4y.prodigy.com|
|3357.1111111111113|  01|   unix.adeptcom.com|
|          23000.25|  01|port42.ts2.msstat...|
| 6676.714285

#### 9. Mostrar una lista de 40 endpoints distintos que generan un código respuesta = 404.

In [109]:
df_data10 = sqlContext.sql("""
    SELECT DISTINCT endpoint, codigo FROM data WHERE codigo = 404  LIMIT 40
""")
df_data10.show()

+--------------------+------+
|            endpoint|codigo|
+--------------------+------+
|/history/apollo/a...|   404|
|/history/apollo/a...|   404|
|/shuttle/resource...|   404|
|/history/apollo/a...|   404|
|/elv/DELTA/uncons...|   404|
|/pub/winvn/readme...|   404|
|/history/apollo/a...|   404|
|/history/history.htm|   404|
|     /sts-71/launch/|   404|
|/history/apollo/a...|   404|
|/www/software/win...|   404|
|/history/apollo/a...|   404|
|/pub/winvn/releas...|   404|
+--------------------+------+



#### 10. Mostrar el top 25 de endpoints que más códigos de respuesta 404 generan.

In [116]:
df_data11 = sqlContext.sql("""
    SELECT endpoint, COUNT (codigo) FROM data WHERE codigo=404 GROUP BY endpoint ORDER BY COUNT(codigo) DESC LIMIT 25
""")
df_data11.show()

+--------------------+-------------+
|            endpoint|count(codigo)|
+--------------------+-------------+
|/pub/winvn/releas...|            4|
|/history/apollo/a...|            4|
|/history/apollo/a...|            2|
|/history/apollo/a...|            2|
|/pub/winvn/readme...|            2|
|/elv/DELTA/uncons...|            1|
|     /sts-71/launch/|            1|
|/history/apollo/a...|            1|
|/shuttle/resource...|            1|
|/history/apollo/a...|            1|
|/history/apollo/a...|            1|
|/www/software/win...|            1|
|/history/history.htm|            1|
+--------------------+-------------+



#### 11. El top 5 de días que se generaron código de respuesta 404.

In [129]:
df_data12 = sqlContext.sql("""
    SELECT COUNT(codigo), date FROM data WHERE codigo=404 GROUP BY date ORDER BY COUNT(codigo) DESC LIMIT 5
""")
df_data12.show()

+-------------+----+
|count(codigo)|date|
+-------------+----+
|           22|  01|
+-------------+----+



In [130]:
sc.stop()